# **Réseaux de transports à la Réunion**

## **Parsing GTFS real-time TripUpdate messages**

In [1]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
import requests

### TripDescriptor

In [2]:
def parse_feed_trip(feed_entity):
    trips = dict()
    trip = {
        "trip_id" : feed_entity.trip_update.trip.trip_id,
        "route_id" : feed_entity.trip_update.trip.route_id,
        "direction_id" : feed_entity.trip_update.trip.direction_id,
        "start_time" : feed_entity.trip_update.trip.start_time,
        "start_date" : feed_entity.trip_update.trip.start_date,
        "schedule_relationship" : feed_entity.trip_update.trip.schedule_relationship
    }
    trips[entity.trip_update.trip.trip_id] = trip
    return (trip)
    
    

### VehicleDescriptor

In [3]:
def parse_feed_vehicle(entity):
    vehicle = {
        "vehicle_id" : entity.trip_update.vehicle.id,
        "vehicle_label" : entity.trip_update.vehicle.label,
        "vehicle_license_plate " : entity.trip_update.vehicle.license_plate
    }
    return (vehicle)

### StopTimeUpdate 

In [4]:
def parse_feed_stop_time_update(entity):
    stop_time_update = {
        "stop_sequence" : entity.trip_update.StopTimeUpdate.stop_sequence,
        "stop_id" : entity.trip_update.StopTimeUpdate.stop_id,
        "arrival": entity.trip_update.StopTimeUpdate.arrival,
        "departure":entity.trip_update.StopTimeUpdate.departure,
        "schedule_relationship" : entity.trip_update.StopTimeUpdate.schedule_relationship
    }
    return (stop_time_update)

## **Parsing GTFS VehiclePosition messages**

### Position

In [5]:
def parse_feed_position(entity):
    position = {
        "latitude" : entity.vehicle.position.latitude,
        "longitude" : entity.vehicle.position.longitude,
        "bearing" : entity.vehicle.position.bearing
    }
    return (position)

## Position en temps réele des bus alterneo 

In [6]:
vehicles = dict()
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://proxy.transport.data.gouv.fr/resource/alterneo-civis-gtfs-rt-vehicle-position')

feed.ParseFromString(response.content)
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = {
            "position":parse_feed_position(entity)
        }
    vehicles[entity.id] = vehicle

import folium
from folium.plugins import MarkerCluster

all_vehicles = [v for v in vehicles.values()]
vehicles_position = pd.DataFrame([v.get("position") for v in all_vehicles])
map = folium.Map([-21.134834,55.474254], zoom_start = 14)

# Ajout des positions trouvées sur la carte
def create_folium_marker(x):
    return folium.Marker(x)

folium_markers = [create_folium_marker((lat,long)) for lat,long in zip(vehicles_position["latitude"],vehicles_position["longitude"])]

for fm in folium_markers:
    fm.add_to(map)
map

# Trip_update car jaune

In [12]:
trips = dict()
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('http://gieactiv.locbusspec.com:9404/api/GtfsRt/GetTripUpdates')

feed.ParseFromString(response.content)
for entity in feed.entity:
    if entity.HasField('trip_update'):
        trip = {
            "trip":parse_feed_trip(entity),
            "vehicle":parse_feed_vehicle(entity)
        }
        trips[entity.id] = trip
        
all_trips = [trips.values() for trip in trips]

In [13]:
all_buses = pd.DataFrame([t.get("vehicle") for t in trips.values()])
all_trips = pd.DataFrame([t.get("trip") for t in trips.values()])
all_stop_time_update = pd.DataFrame([t.get("stop-time-update") for t in trips.values()])

final = pd.concat([all_buses,all_trips],axis=1)
final

,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,route_id,direction_id,start_time,start_date,schedule_relationship
0,307,307,FA-340-YM,24235-387,18,0,11:00:00,20230227,0
1,305,305,DQ-732-AL,24236-387,18,1,11:00:00,20230227,0
2,709,709,DQ-125-PH,24239-387,18,0,12:00:00,20230227,0
3,716,716,EK-351-MV,24440-387,19,1,10:30:00,20230227,0
4,723,723,FK-453-XG,24447-387,19,1,11:30:00,20230227,0
5,501,501,EK-379-AY,24845-387,21,1,05:20:00,20230227,0
6,615,615,DQ-095-EJ,24853-387,21,1,06:20:00,20230227,0
7,105,105,EM-557-EY,24876-387,21,1,08:40:00,20230227,0
8,502,502,DR-104-GY,24886-387,21,0,10:00:00,20230227,0
9,210,210,DP-838-NQ,24894-387,21,0,11:00:00,20230227,0
